# Prepare the dataset

In [1]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train':'train.csv', 'validation':'validation.csv', 'test':'test.csv'})

Found cached dataset csv (C:/Users/Omar/.cache/huggingface/datasets/csv/default-f3637b5a9b9def03/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
dataset["train"][0]

{'text': 'يعزا تراجع إيرادات ياهو بشكل كبير إلى الانخفاض الحاد في نشاط الإعلانات المرئية الرقمية وتساعد فلاري مطوري التطبيقات على تحليل البيانات المتعلقة بمستخدميهم وتوفير إعلانات تناسب أذواقهم. واشترى بعض منافسي ياهو شركات تعمل في تكنولوجيا إعلانات الهواتف المحمولة في محاولة لجذب المسوقين إلى تطبيقاتها وتحقيق إيرادات من الإعلانات على تطبيقات شركات أخرى. ولم تكشف الشركتان عن البنود المالية لصفقة الاستحواذ. لكن بعض التقارير أشارت إلى أن ياهو دفعت ما بين 200 و300 مليون دولار للاستحواذ على فلاري، لتكون بذلك واحدة من أكبر عمليات الاستحواذ لياهو منذ أن استحوذت على منصة تدوين "تومبير" عام 2012. مواضيع قد تهمك نهاية تعزيز الإيرادات وتأتي صفقة ياهو للاستحواذ على فلاري بعد أيام فقط من إعلان تراجع أرباحها بواقع 18 في المئة إلى 270 مليون دولار خلال الأشهر الثلاثة حتى نهاية يونيو/حزيران الماضي. وتراجعت الإيرادات أيضا في شركة الانترنت العملاقة بواقع ثلاثة في المئة إلى 1.08 مليار دولار. ويعزا هذا التراجع بشكل كبير إلى الانخفاض الحاد في نشاط الإعلانات المرئية الرقمية، التي تراجعت بواقع ثمانية في المئ

In [3]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from arabert.preprocess import ArabertPreprocessor
from transformers import AutoTokenizer

# Preprocess the dataset

In [4]:
model_name = "aubmindlab/bert-base-arabertv2"
preprocessor = ArabertPreprocessor(model_name=model_name)

[2023-07-03 16:25:51,227 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [5]:
def preprocess_function(text):
  return preprocessor.preprocess(text)

import re

def unpreprocess_function(text):
  temp = preprocessor.unpreprocess(text)
  temp = re.sub('\[CLS\]|\[PAD\]|\[SEP\]', '', temp)
  return re.sub('\s+$|^\s', '', temp)

# Load the tokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained("./AraT5")
model = AutoModelForSeq2SeqLM.from_pretrained("./AraT5")

# Encode the dataset with the tokenizer

In [10]:
max_input_length = 1024
max_target_length = 128

def tokenize_function(examples):
  
  inputs = [preprocess_function(text) for text in examples['text']]
  model_inputs = tokenizer(inputs, padding='max_length', truncation=True)

  labels = tokenizer(examples['summary'], padding='max_length', max_length=128, truncation=True, return_tensors="pt")

  model_inputs['labels'] = labels['input_ids']
  return model_inputs


In [11]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
small_test_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))

Map:   0%|          | 0/60878 [00:00<?, ? examples/s]

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x96 in position 85: invalid start byte

In [9]:
import sys
print(sys.getdefaultencoding())

utf-8


In [12]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Load metrics

In [13]:
import evaluate
import numpy as np

rouge = evaluate.load('rouge')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# import torch
# torch.cuda.is_available()

False

# Train the model

In [14]:
training_args = Seq2SeqTrainingArguments(
    output_dir="finetuned-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True
    # fp16=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

NameError: name 'small_train_dataset' is not defined

In [ ]:
trainer.train()